#   1. Import Librarys

In [0]:
# %pip install yfinance



In [0]:
import requests
import pandas as pd
from datetime import datetime


#  2. Extract and Transform data

In [0]:
def Extract_Bitcoin_Data():
  url = "https://api.coinbase.com/v2/prices/spot"
  response = requests.get(url)
  data = response.json()
  return data

In [0]:
def extrair_cotacao_usd_brl():
    """Extrai a cotação USD-BRL da API CurrencyFreaks."""
    api_key = 'dbd4b4b2abe6490ebb912bb254a9054a'
    url = f'https://api.currencyfreaks.com/v2.0/rates/latest?apikey={api_key}'
    resposta = requests.get(url)
    return resposta.json()


In [0]:
def tratar_dados_bitcoin(dados_json, taxa_usd_brl):
    """Transforma os dados brutos da API, renomeia colunas, adiciona timestamp e converte para BRL."""
    valor_usd = float(dados_json['data']['amount'])
    criptomoeda = dados_json['data']['base']
    moeda_original = dados_json['data']['currency']
    
    # Convertendo de USD para BRL
    valor_brl = valor_usd * taxa_usd_brl
    
    # Adicionando timestamp como datetime object
    timestamp = datetime.now()
    
    dados_tratados = [{
        "valor_usd": valor_usd,
        "valor_brl": valor_brl,
        "criptomoeda": criptomoeda,
        "moeda_original": moeda_original,
        "taxa_conversao_usd_brl": taxa_usd_brl,
        "timestamp": timestamp
    }]
    
    return dados_tratados

In [0]:
# Extraindo dados
Dados_bitcoin = Extract_Bitcoin_Data()
dados_cotacao = extrair_cotacao_usd_brl()

# Extraindo a taxa de conversão USD-BRL
taxa_usd_brl = float(dados_cotacao['rates']['BRL'])

# Tratando os dados e convertendo para BRL
dados_bitcoin_tratado = tratar_dados_bitcoin(Dados_bitcoin, taxa_usd_brl)


print(dados_bitcoin_tratado)

# 3. Config Unity Catalog

In [0]:
%sql

-- TODO: Create the catalog in Unity Catalog
-- Name: pipeline_api_bitcoin
-- COMMENT: 'Demo catalog created for the pipeline_api_bitcoin workshop'



--CREATE CATALOG IF NOT EXISTS pipeline_api_bitcoin
--COMMENT 'Catalog for pipeline api Biticoin';


In [0]:
%sql
--TODO: Create the Lakehouse schema
-- Name: pipeline_api_bitcoin.lakehouse
-- Comment: 'Lakehouse schema to store processed data'

--CREATE SCHEMA IF NOT EXISTS pipeline_api_bitcoin.lakehouse
--COMMENT 'Lakehouse schema to store processed data';


In [0]:

%sql
--TODO: Create the Lakehouse schema
-- Name: pipeline_api_bitcoin.lakehouse
-- Comment: 'Lakehouse schema to   store delta data'

--CREATE SCHEMA IF NOT EXISTS pipeline_api_bitcoin.biticoin_delta
--COMMENT 'Lakehouse schema to store delta data';

In [0]:
%sql
-- TODO: Create the volume for raw files
-- Name: pipeline_api_bitcoin.lakehouse.raw_files
-- Comment: 'Volume for raw files from initial ingestion'

--CREATE VOLUME IF NOT EXISTS pipeline_api_bitcoin.lakehouse.raw_files
--COMMENT 'Volume for raw files from initial ingestion';


# 4. Creating a Pandas DataFrame




In [0]:
# # Create a Pandas DataFrame from the processed data

df_bitcoin = pd.DataFrame(dados_bitcoin_tratado)

# Show Dataframe
display(df_bitcoin)


# 5. Saving to JSON using Pandas


In [0]:
# # TODO: Save DataFrame as JSON
# # 1. Get the timestamp from the first record: dados_bitcoin_tratado[0]["timestamp"]
# event_ts = dados_bitcoin_tratado[0]["timestamp"]

# # 2. Convert it to a file-friendly format: .strftime("%Y%m%d_%H%M%S_%f")
# ts = event_ts.strftime("%Y%m%d_%H%M%S_%f")

# # 3. Create the path: f"/Volumes/pipeline_api_bitcoin/lakehouse/raw_files/bitcoin_{ts}.json"
# json_file = f"/Volumes/pipeline_api_bitcoin/lakehouse/raw_files/bitcoin_{ts}.json"

# # 4. Save using Pandas
# df_bitcoin.to_json(json_file, orient='records', date_format='iso', indent=2)
# print(f"DataFrame saved to {json_file}")

# 6. Saving to Parquet using Pandas

In [0]:
# # TODO: Save DataFrame as Parquet
# # 1. Create the path: f"/Volumes/pipeline_api_bitcoin/lakehouse/raw_files/bitcoin_{ts}.parquet"
# parquet_file = f"/Volumes/pipeline_api_bitcoin/lakehouse/raw_files/bitcoin_{ts}.parquet"


# # 2. Save using: df.write.mode("overwrite").parquet(parquet_path)
# df_bitcoin.to_parquet(parquet_file, index=False)
# print(f"DataFrame saved to {parquet_file}")



#   8. Saving as a Delta Table
# 
##   🎯 What is a Delta Table?
# 
###  **Delta Lake** is an open-source storage layer that brings **ACID transactions** and **time travel** to Data Lakes. Delta Tables are tables that use the Delta format.
# 
##  **Technical Features:**
####  ✅ **ACID Transactions**: Ensures data consistency (Atomicity, Consistency, Isolation, Durability)
####   ✅ **Time Travel**: Allows access to previous versions of the data
####  ✅ **Schema Enforcement**: Validates and enforces the defined schema
####   ✅ **Upsert/Merge**: Supports efficient update and merge operations
####   ✅ **Parquet-based**: Uses Parquet as the physical format (inherits all its benefits)
####   ✅ **Transaction Log**: Maintains a log of all transactions (Delta Log)
####   ✅ **Optimize & Z-Order**: Tools to optimize performance
#### 
####  **Advantages over plain Parquet:**
####   ✅ **ACID Transactions**: Multiple concurrent writes without corruption
####   ✅ **Time Travel**: Access historical versions of the data
####   ✅ **Schema Evolution**: Allows the schema to evolve over time
####   ✅ **Delete/Update**: Supports update and delete operations
####   ✅ **Auditing**: Complete history of changes
####   ✅ **Performance**: Automatic optimizations (compaction, indexing)


In [0]:
# Convert a Pandas DataFrame to a PySpark DataFrame
# type: ignore - spark is available in the Databricks environment
df_bitcoin_spark = spark.createDataFrame(df_bitcoin)

# Show schema
df_bitcoin_spark.printSchema()

display(df_bitcoin_spark)


In [0]:
# TODO: Save DataFrame as a Delta Table
# 1. Define the path: "pipeline_api_bitcoin.bitcoin_data.bitcoin_data"
delta_table_path = "pipeline_api_bitcoin.biticoin_delta.btc_data"
# 2. Save using: df.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(delta_table_path)


df_bitcoin_spark.write \
    .format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable(delta_table_path)


#  10. Converting a Delta Table to a DataFrame


In [0]:
# Read a Delta Table as a Spark DataFrame
# type: ignore - spark is available in the Databricks environment
df_delta = spark.read.table(delta_table_path)  

# Show data
df_delta.show(truncate=False)

# Querying a Delta Table with SQL


In [0]:
%sql

 SELECT *
  FROM pipeline_api_bitcoin.biticoin_delta.btc_data
 ORDER BY timestamp DESC
 LIMIT 10

#  Checking the Delta Table history (Time Travel)


In [0]:
%sql
DESCRIBE HISTORY pipeline_api_bitcoin.biticoin_delta.btc_data

#   11. Pipeline Summary
# 
###  This complete pipeline performs:
# 
####  1. ✅ **Extraction**: Retrieves data from the Coinbase API and the USD-BRL exchange rate
####  2. ✅ **Transformation**: Cleans the data, converts values to BRL, and adds a timestamp
####  3. ✅ **Infrastructure**: Creates the Catalog and Lakehouse Schema in Unity Catalog
####  4. ✅ **Load in multiple formats using Pandas**:
####       - **JSON**: Readable text format, ideal for raw data (saved with Pandas)
####       - **Parquet**: Columnar binary format, optimized for Big Data (saved with Pandas)
####  5. ✅ **Pandas → PySpark conversion**: Converts a Pandas DataFrame to PySpark
####  6. ✅ **Delta Table**: Saves data as a Delta Table using PySpark (format with ACID transactions and time travel)
####  7. ✅ **Conversion**: Delta Table → Spark DataFrame
###
#### **Format Comparison:**
#### - 📄 **CSV**: Readable, larger, slower → Use for debugging and small datasets
####   - 📊 **Parquet**: Binary, smaller, faster → Use for Big Data and analytics
####   - 🎯 **Delta Table**: Parquet + ACID + Time Travel → Use for Data Warehouses and critical pipelines
